In [ ]:
#!pip install nba_api

I would like to create some visualizations that can give someone an idea of how a game was played at a glance using NBA py package

In [4]:
import pandas as pd
import nba_api as nba
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playbyplay
from nba_api.stats.endpoints import scoreboard
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoreusagev2
from nba_api.stats.endpoints import boxscoretraditionalv2 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
sns.set(style="whitegrid")
# Basic Request
#player_info = commonplayerinfo.CommonPlayerInfo(player_id=2544)

In [5]:
player_info.player_headline_stats.get_data_frame()

NameError: name 'player_info' is not defined

In [6]:
boxadv=boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=gameid).player_stats.get_data_frame().fillna(0)
boxadv.head()
box = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=gameid).player_stats.get_data_frame().fillna(0)
box.head()

NameError: name 'gameid' is not defined

In [ ]:
def minute_conversion(minutes):
    return int(minutes.split(':')[0])+int(minutes.split(':')[1])/60


In [7]:
usage = boxadv[['TEAM_ID','TEAM_ABBREVIATION','PLAYER_NAME','MIN','USG_PCT']]
usage=usage[usage['MIN']!=0]
usage_raps=usage[(usage['TEAM_ID']==1610612761)]
usage_raps.loc[:,'MIN']=usage_raps.loc[:,'MIN'].apply(minute_conversion)
usage_raps

NameError: name 'boxadv' is not defined

In [8]:
pts = box[['TEAM_ID','TEAM_ABBREVIATION','PLAYER_NAME','MIN','PTS']]
pts_raps = pts[(pts['TEAM_ID']==1610612761) & (pts['MIN']!=0) ]
pts_raps

sns.set_color_codes("pastel")
sns.barplot(x="PTS", y="PLAYER_NAME", data=pts_raps,
            label="Total", color="b")
sns.grid(False)

NameError: name 'box' is not defined

In [6]:
now = datetime.datetime.now()+ dt.timedelta(days=-2)
league_scores = scoreboard.Scoreboard(game_date=now).game_header.get_data_frame()
#Raps Team_ID = 1610612761
gameid = league_scores.loc[(league_scores['HOME_TEAM_ID']==1610612761) | (league_scores['VISITOR_TEAM_ID']==1610612761),'GAME_ID'].values[0]
pbp = playbyplay.PlayByPlay(game_id=gameid).play_by_play.get_data_frame()
game_score = pbp.loc[pbp['EVENTMSGTYPE']==1, ['PCTIMESTRING','PERIOD','SCOREMARGIN']].copy()
#clean up the data
game_score.loc[game_score['SCOREMARGIN']=='TIE','SCOREMARGIN']=0
game_score=game_score.fillna(0)
#ax = sns.lineplot(x="PCTIMESTRING", y="SCOREMARGIN", data=score)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
pbp['PERIOD'].max()

In [ ]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    print(response)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [ ]:
proxies = get_proxies()
print(proxies)
#boxscore_summary = game.BoxscoreSummary("0021600457")
#print(boxscore_summary.game_summary())

In [ ]:
#Score difference by minute
#Line chart with score difference. Bottom of chart is home team, top of chart is away team.

In [ ]:
#Usage Rate + Minutes - spider chart? Radar Chart?

In [ ]:
#Points overlayed on PPG per player, 
#maybe a horizintal bar chart with top X players and their points in the game
#Expand to Rebounds and other key stats

In [ ]:
#Pace

In [ ]:
#True Shooting %

In [ ]:
#Network of assists?